# 04 — Neural Models

Train CNN, BiLSTM, and Transformer classifiers on one-hot encoded sequences.

In [ ]:
import sys
sys.path.insert(0, "../src")
from data_loading import build_dataset, DataSplitter
from feature_extractors import build_onehot_pipeline
from neural_models import build_all_neural_models
from evaluation import ModelEvaluator
from utils import load_config, set_global_seed
import numpy as np

config = load_config("../configs/default.yaml")
set_global_seed(42)
dataset = build_dataset("../data/class_a.fasta", "../data/class_b.fasta", config)

In [ ]:
splitter = DataSplitter(test_size=0.2, val_size=0.1, seed=42)
train_ds, val_ds, test_ds = splitter.split(dataset)

oh_pipeline = build_onehot_pipeline(config)
X_train = oh_pipeline.fit_transform(train_ds.sequences)
X_val = oh_pipeline.transform(val_ds.sequences)
X_test = oh_pipeline.transform(test_ds.sequences)
seq_length = config["features"]["one_hot"]["max_length"]
print(f"One-hot shape: {X_train.shape}")

In [ ]:
cw = dataset.class_weights
cw_array = [cw[i] for i in sorted(cw)]
trainers = build_all_neural_models(config, seq_length=seq_length, n_classes=2, class_weights=cw_array)
for t in trainers:
    print(f"
Training {t.name}...")
    t.fit(X_train, train_ds.labels, X_val, val_ds.labels)
    print(f"  {t}")

In [ ]:
evaluator = ModelEvaluator(output_dir="../results/plots", label_names=dataset.label_names)
for t in trainers:
    y_pred = t.predict(X_test)
    y_proba = t.predict_proba(X_test)
    evaluator.evaluate_model(t.name, test_ds.labels, y_pred, y_proba)

print(evaluator.comparison_table()[["Model", "accuracy", "f1_weighted", "roc_auc"]].to_string(index=False))